###Getting Screens Data

In [291]:
import requests
from bs4 import BeautifulSoup
import pickle
import time
from collections import defaultdict
import re
import os

In [292]:
os.getcwd()
os.chdir('/Users/ItelinaMa/Documents/Metis/Luther')

In [293]:
urllist = ['http://www.boxofficemojo.com/daily/?view=bymovie&yr=2015&page=1&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2015&page=2&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2014&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2014&page=2&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2014&page=3&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2013&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2013&page=2&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2013&page=3&sort=title&order=ASC&p=.htm','http://www.boxofficemojo.com/daily/?view=bymovie&yr=2012&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2012&page=2&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2012&page=3&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2011&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2011&page=2&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2011&page=3&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2010&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2010&page=2&sort=title&order=ASC&p=.htm', 'http://www.boxofficemojo.com/daily/?view=bymovie&yr=2010&page=3&sort=title&order=ASC&p=.htm']

In [294]:
def getSoups(urllist):
    soups = []
    for url in urllist :
        response = requests.get(url)
        while response.status_code != 200:
            time.sleep(2)
            response = requests.get(url)
        soup = BeautifulSoup(response.text)
        soups.append(soup)
    return soups

In [295]:
soups = getSoups(urllist )

#####Find Production Budget
Note: BoxOfficeMojo does not have very complete budget data so I am moving to numbers.com for that

In [296]:
def getLinks(soups=soups):
    movies = []
    links = []
    for soup in soups:
        for i in range(1, len(soup.find_all('table')[2].find_all('tr'))):
            link = soup.find_all('table')[2].find_all('tr')[i].find_all('a')[0]['href']
            newlink = 'http://www.boxofficemojo.com'+link.replace('daily', 'main', 1)
            links.append(newlink)
            movies.append(soup.find_all('table')[2].find_all('tr')[i].find_all('a')[0].text)
    return movies, links

In [297]:
movies, links = getLinks(soups)
len(movies), len(links)

(1520, 1520)

In [298]:
def getBudget(movies=movies, links=links):
    budget = defaultdict(list)
    for i, link in enumerate(links):
        response = requests.get(link)
        while response.status_code != 200:
            time.sleep(2)
            response = requests.get(url)
        #print response.status_code
        #print link
        soup = BeautifulSoup(response.text)
        #print movies[i]
        atext = re.compile('Production Budget:')
        sstring = soup.find(text=atext)
        if not sstring:
            budget[movies[i]] = 'NA'
        else:
            budget[movies[i]] = sstring.findNextSibling().text
    return budget

In [299]:
budget = getBudget()

In [300]:
with open('budget.pkl', 'w') as picklefile:
    pickle.dump(budget, picklefile)

In [312]:
def getScreens(movies=movies, links=links):
    screens = {}
    for i, link in enumerate(links):
        response = requests.get(link)
        while response.status_code != 200:
            time.sleep(2)
            response = requests.get(url)
        soup = BeautifulSoup(response.text)
        if len(soup.find_all('table'))<8:
            print movies[i]
            screens[movies[i]] = 'NA'
        elif soup.find_all('table')[7].find_all(class_='mp_box')[1].find_all(text=re.compile('theaters')) == []:
            screens[movies[i]] = 'NA'
        else:
            print movies[i]
            screens[movies[i]] = unicode(soup.find_all('table')[7].find_all(class_="mp_box")[1].find_all(text=re.compile('theaters'))[-1]).encode('ascii','ignore')
    return screens

In [311]:
url = 'http://www.boxofficemojo.com/movies/?page=main&id=barfi.htm'
response = requests.get(url)
response.status_code
soup = BeautifulSoup(response.text)
soup.find_all('table')[7].find_all(class_='mp_box')[1].find_all(text=re.compile('theaters')) == []

False

In [313]:
screens = getScreens()

The 100-Year Old Man Who Climbed Out the Window and Disappeared
3 Hearts
5 Flights Up
'71
ABCD (Any Body Can Dance) 2
Adult Beginners
After The Ball
The Age of Adaline
Aloha
Americons
Animals (2015)
The Apu Trilogy
Avengers: Age of Ultron
Baahubali: The Beginning
Batkid Begins
Beyond the Mask
Beyond the Reach
Big Game
Black or White
Black Sea
Black Souls (Anime Nere)
Blackbird
Blackhat
The Boy Next Door
Brotherly Love
Buen Dia, Ramon
Cake
Chappie
Child 44
Chocolate City
Cinderella (2015)
The Connection
The D Train
Danny Collins
Dark Star: H.R. Giger's World
Deli Man
Desert Dancer
Detective Byomkesh Bakshy
The Devil's Violinist
Digging Up the Marrow
The Divergent Series: Insurgent
Do You Believe?
Don't Think I've Forgotten: Cambodia's Lost Rock and Roll
Dope
The DUFF
Entourage
Escobar: Paradise Lost
Eva
Ex Machina
Faith of Our Fathers
Far from the Madding Crowd
Fifty Shades of Grey
The Film Critic
Focus (2015)
Forbidden Games (1952)
Freetown
Furious 7
The Gallows
Game of Thrones (IMAX)


In [314]:
screens['X-Men: Days of Future Past']

'4,001 theaters'

In [315]:
len(screens)

1520

In [316]:
with open('screens.pkl', 'w') as picklefile:
    pickle.dump(screens, picklefile)